In [1]:
import pandas as pd
from pyxlsb import open_workbook
import io
import csv
import teradatasql
import numpy as np
from datetime import datetime
import locale
import pandas as pd
from pyxlsb import open_workbook
import io
import csv
from datetime import datetime
import os
from os.path import join
import re
import teradatasql
import numpy as np

class SkipRowsError(Exception):
    pass

class ReportMonthError(Exception):
    pass
locale.setlocale(locale.LC_ALL, "ru")

'ru'

In [48]:
path = r'\\corp.tele2.ru\\plm_cluster\\All\\_Плановый Sales mix\\'
for file in enumerate(os.listdir(path)):
    print(file)
file_index = int(input('Какой номер нужен? '))
shortname = None
for i, file in enumerate(os.listdir(path)):
    if i == file_index:
        print(f'Выбран {file}')
        shortname = file
        filename = path + shortname
        break
if not shortname:
    print('Файл не выбран')

(0, '2019')
(1, '2020')
(2, 'for Tableau')
(3, 'SalesMix_апр21.xlsx')
(4, 'SalesMix_июн21.xlsx')
(5, 'SalesMix_май21.xlsx')
(6, 'SalesMix_март21 — копия.xlsx')
(7, 'SalesMix_март21.xlsx')
(8, 'Архив')
(9, 'исходник')
(10, 'Новая папка')
(11, 'Расчет целевого Sales mix_фев21.xlsb')
(12, 'Расчет целевого Sales mix_январь.xlsb')
(13, 'Расчет целевого Sales mix_январь_20_исправления.xlsb')
Какой номер нужен? 13
Выбран Расчет целевого Sales mix_январь_20_исправления.xlsb


In [49]:
with open_workbook(filename) as wb:
    output = io.StringIO()
    writer = csv.writer(output, quoting=csv.QUOTE_NONNUMERIC)
    with wb.get_sheet('Channels_MIX') as sheet:
        for row in sheet.rows():
            csv_line = [r.v for r in row]
            writer.writerow(csv_line)
    output.seek(0)
csvreader = csv.reader(output)

In [50]:
# output.seek(0)
# row1 = next(csvreader)
# try:
#     ordinal_days = int(float(row1[0]))
# except:
#     raise ReportMonthError
# report_month_dt = datetime.fromordinal(datetime(1900, 1, 1).toordinal() + ordinal_days - 2).date()
# print(f'Отчетный месяц: {report_month_dt}')

In [51]:
report_month_str = input('Введите месяц в формате yyyy-mm-dd?\n')
report_month_dt = datetime.strptime(report_month_str, '%Y-%m-%d')

Введите месяц в формате yyyy-mm-dd?
2021-01-01


In [52]:

for row_number, row in enumerate(csvreader):
    if row[0] == 'Регион':
        col_row = row_number
        break
else:
    raise SkipRowsError
col_indexes = [i for i in range(2)]  # столбцы
for index, cell_value in enumerate(row):
    try:
        if 'план' in cell_value: col_indexes.append(index)
    except:
        continue
print(f'Старые столбцы: {[row[index] for index in col_indexes]}')

Старые столбцы: ['Регион', 'Channels', 'Классический  план', 'Мой Tele2  план', 'Мой разговор  план', 'Мой онлайн  план', 'Мой онлайн +  план', 'Безлимит  план', 'Везде онлайн  план', 'Везде онлайн+  план', 'Other  план']


In [53]:
output.seek(0)
sm = pd.read_csv(output, skiprows=col_row)
sm1 = sm.iloc[:, col_indexes].dropna()
sm1

,Регион,Channels,Классический план,Мой Tele2 план,Мой разговор план,Мой онлайн план,Мой онлайн + план,Безлимит план,Везде онлайн план,Везде онлайн+ план,Other план
0,Владимир,Total,1407.840206,373.762887,3690.285567,4442.794845,1250.859794,451.007216,139.538144,0.0,328.911340
8,Волгоград,Total,1926.259600,327.350600,5865.820000,5766.724426,2855.059117,477.979486,980.159600,0.0,722.647171
16,Вологда,Total,1096.680713,430.613250,2961.938117,3506.751433,1010.774575,229.897864,0.000000,0.0,118.344048
24,Йошкар-Ола,Total,515.872733,18.252480,1291.122819,788.699282,2687.917893,180.603490,0.000000,0.0,133.531303
32,Калининград,Total,791.524371,704.948269,2601.534525,2534.476817,586.431493,164.875308,0.000000,0.0,219.209216
...,...,...,...,...,...,...,...,...,...,...,...
456,Саратов,Total,1353.204261,0.000000,5889.027863,5717.907191,2473.369187,613.557673,0.000000,0.0,437.933825
464,Сахалин,Total,282.350332,39.530000,596.903000,909.190000,1517.952000,86.966000,395.300000,0.0,124.808668
472,Чебоксары,Total,842.931286,0.000000,1847.946807,2204.451681,5929.246874,246.985621,0.000000,0.0,211.437731
480,Иваново,Total,242.096437,0.000000,1038.062285,2744.546929,5401.702948,74.400369,0.000000,0.0,112.191032


In [54]:
col_names = {
    'Регион': 'region',
    'Channels': 'channel',
    'МР': 'macroregion',
    'Кластер': 'cluster',
    'Мой безлимит': 'Безлимит',
    'Мой онлайн +': 'Мой онлайн+',
    'Мой Tele2': 'Лайт/Мой Tele2',
    'Лайт': 'Лайт/Мой Tele2',
    '*любое старое название': '*любое новое название',

}
sm1 = sm1.rename(columns=lambda x: re.sub(r'(.+)\s*план', r'\1', x).strip()).rename(columns=col_names)
sm1 = sm1.rename(columns=lambda x: re.sub(r'(.+)\s*план', r'\1', x).strip()).rename(columns=col_names)
sm1.insert(0, 'report_month', report_month_dt)
print(f'Новые столбцы: {[sm1.columns.to_list()]}')

Новые столбцы: [['report_month', 'region', 'channel', 'Классический', 'Лайт/Мой Tele2', 'Мой разговор', 'Мой онлайн', 'Мой онлайн+', 'Безлимит', 'Везде онлайн', 'Везде онлайн+', 'Other']]


In [55]:
fixed_cols_count = 3
sm2 = sm1.melt(id_vars=sm1.columns[:fixed_cols_count].to_list(),
               value_vars=sm1.columns[fixed_cols_count:].to_list(),
               var_name='tariff',
               value_name='subs_count')
sm2 = sm2.loc[(sm2.channel == 'Total'),:].dropna()
sm2.head()

,report_month,region,channel,tariff,subs_count
0,2021-01-01,Владимир,Total,Классический,1407.840206
1,2021-01-01,Волгоград,Total,Классический,1926.259600
2,2021-01-01,Вологда,Total,Классический,1096.680713
3,2021-01-01,Йошкар-Ола,Total,Классический,515.872733
4,2021-01-01,Калининград,Total,Классический,791.524371


In [56]:
with teradatasql.connect() as session:
    query = '''
        sel
            case when name_report in
                ('Классический','Мой разговор','Везде онлайн','Везде онлайн+','Лайт/Мой Tele2','Мой онлайн','Мой онлайн+','Безлимит')
            then name_report
            else 'Other'
            end as name_report,
            min(tp_id)
        from PRD2_DIC_V.PRICE_PLAN
        group by 1;

    '''
    price_plan_df = pd.read_sql(query, session)
price_plan_df

,name_report,TP_ID
0,Классический,10168.0
1,Везде онлайн+,20016532.0
2,Мой разговор,10623.0
3,Везде онлайн,14093.0
4,Лайт/Мой Tele2,10624.0
5,Безлимит,15960.0
6,Мой онлайн,10628.0
7,Мой онлайн+,10630.0
8,Other,-912.0


In [57]:
report_month_dt = sm2.report_month.unique()[0]
if not report_month_dt:       
    while 1:
        report_month_str = input('Какой месяц в формате yyyy-mm-dd?')
        try:
            report_month_dt = datetime.strptime(report_month_str, '%Y-%m-%d')
        except ValueError:
            print('Некорректный формат. Попробуйте еще.')
        else:
            break
                

union_df_with_tp_id = pd.merge(sm2, price_plan_df, how='inner', left_on='tariff', right_on='name_report')
union_df_with_tp_id['report_month'] = report_month_dt
union_df_with_tp_id['param_1'] = 'SM Plan'
union_df_with_tp_id.head()

,report_month,region,channel,tariff,subs_count,name_report,TP_ID,param_1
0,2021-01-01,Владимир,Total,Классический,1407.840206,Классический,10168.0,SM Plan
1,2021-01-01,Волгоград,Total,Классический,1926.259600,Классический,10168.0,SM Plan
2,2021-01-01,Вологда,Total,Классический,1096.680713,Классический,10168.0,SM Plan
3,2021-01-01,Йошкар-Ола,Total,Классический,515.872733,Классический,10168.0,SM Plan
4,2021-01-01,Калининград,Total,Классический,791.524371,Классический,10168.0,SM Plan


In [58]:
with teradatasql.connect() as session:
    query = '''
        sel DISTINCT
                min(b.branch_id) as branch_id,
                r.region_name
        from PRD2_DIC_V.BRANCH b
        inner join PRD2_DIC_V.REGION r
            on b.region_id=r.region_id
        where product_cluster_name is not null
            and branch_id is not null
            and b.branch_name not like '%CDMA%'
            and b.branch_name not like '%MVNO%'
            and b.branch_name not like '%LTE450%'
            and b.product_cluster_name<>'Deferred'
        group by 2;

    '''
    region_df = pd.read_sql(query, session)
region_dict = {
    'Краснодар':'Краснодар и Адыгея'   , 
    'Салехард':'ЯНАО'    ,    
    'Абакан':'Хакасия'      ,      
    'Чебоксары':'Чувашия'      ,
    'Ростов-на-Дону':'Ростов на Дону' ,
    'Санкт-Петербург':'С.Петербург'  ,
    'Ханты-Мансийск':'ХМАО' ,
    'Нижний Новгород':'Н.Новгород'   , 
    'Великий Новгород':'В.Новгород'  ,
    'Саранск':'Мордовия'     ,
    'Йошкар-Ола':'Марий Эл'    ,
    'Улан-Удэ':'Бурятия'   ,  
}
union_df_with_tp_id.region = union_df_with_tp_id.region.replace(region_dict)
union_df_with_branch_id = pd.merge(union_df_with_tp_id, region_df,
                               how='left', left_on='region', right_on='REGION_NAME')
union_df_with_branch_id.head()

,report_month,region,channel,tariff,subs_count,name_report,TP_ID,param_1,branch_id,REGION_NAME
0,2021-01-01,Владимир,Total,Классический,1407.840206,Классический,10168.0,SM Plan,28.0,Владимир
1,2021-01-01,Волгоград,Total,Классический,1926.259600,Классический,10168.0,SM Plan,64.0,Волгоград
2,2021-01-01,Вологда,Total,Классический,1096.680713,Классический,10168.0,SM Plan,36.0,Вологда
3,2021-01-01,Марий Эл,Total,Классический,515.872733,Классический,10168.0,SM Plan,106.0,Марий Эл
4,2021-01-01,Калининград,Total,Классический,791.524371,Классический,10168.0,SM Plan,22.0,Калининград


In [59]:
na_branch = union_df_with_branch_id[union_df_with_branch_id.branch_id.isna()]
na_branch.region.value_counts()
#если пусто, значит все регионы нашлись - все ок

Series([], Name: region, dtype: int64)

In [60]:
union_df_with_branch_id = union_df_with_branch_id[union_df_with_branch_id.subs_count != '0x7']
union_df_with_branch_id.subs_count = union_df_with_branch_id.subs_count.astype(float)
union_df_with_branch_id.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558 entries, 0 to 557
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   report_month  558 non-null    datetime64[ns]
 1   region        558 non-null    object        
 2   channel       558 non-null    object        
 3   tariff        558 non-null    object        
 4   subs_count    558 non-null    float64       
 5   name_report   558 non-null    object        
 6   TP_ID         558 non-null    float64       
 7   param_1       558 non-null    object        
 8   branch_id     558 non-null    float64       
 9   REGION_NAME   558 non-null    object        
dtypes: datetime64[ns](1), float64(3), object(6)
memory usage: 48.0+ KB


c:\users\igor.i.plotnikov\appdata\local\programs\python\python38\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [61]:
cols = ['report_month', 'param_1', 'NULL8',
        'NULL', 'NULL1', 'BRANCH_ID', 'NULL6',
        'TP_ID', 'NULL3', 'NULL7', 'NULL4', 'subs_count']
df = union_df_with_branch_id.reindex(columns = cols,
                                     fill_value = np.NaN).where((pd.notnull(union_df_with_branch_id)), None)
df.head()

,report_month,param_1,NULL8,NULL,NULL1,BRANCH_ID,NULL6,TP_ID,NULL3,NULL7,NULL4,subs_count
0,2021-01-01,SM Plan,None,None,None,None,None,10168,None,None,None,1407.84
1,2021-01-01,SM Plan,None,None,None,None,None,10168,None,None,None,1926.26
2,2021-01-01,SM Plan,None,None,None,None,None,10168,None,None,None,1096.68
3,2021-01-01,SM Plan,None,None,None,None,None,10168,None,None,None,515.873
4,2021-01-01,SM Plan,None,None,None,None,None,10168,None,None,None,791.524


In [62]:
# чистим product_parameters_test_for_sap и заполняем данными
with teradatasql.connect() as con:
    with con.cursor() as cur:
 
        print('deleting from uat_product.product_parameters...')
        cur.execute(f'''
            delete from UAT_PRODUCT.PRODUCT_PARAMETERS
                where param_1 = 'SM Plan'
                    and (report_date = '{pd.to_datetime(report_month_dt):%Y-%m-%d}'
                        or report_date is null)
        ''')
        print(f'{cur.rowcount} rows deleted from uat_product.product_parameters')    
        batchsize = 100000
        print('inserting into UAT_PRODUCT.PRODUCT_PARAMETERS...')
        for num in range(0, len(df), batchsize):
            cur.executemany(f'''
             INSERT into UAT_PRODUCT.PRODUCT_PARAMETERS ({','.join('?'*len(df.columns))})
            ''',
                [tuple(row) for row in df.iloc[num:num+batchsize,:].itertuples(index=False)]
                )
        print(f'{len(df)} rows inserted into UAT_PRODUCT.PRODUCT_PARAMETERS in total.')

deleting from uat_product.product_parameters...
657 rows deleted from uat_product.product_parameters
inserting into UAT_PRODUCT.PRODUCT_PARAMETERS...
558 rows inserted into UAT_PRODUCT.PRODUCT_PARAMETERS in total.


In [30]:
# newfilename = r'C:\Users\igor.i.plotnikov\Documents\My Tableau Repository\Datasources\TheGreatDashboard\Sales Mix.xlsx'
# sm2.to_excel(newfilename, index = False)